# Random Forest
Decision Trees can suffer from high variance which makes their results fragile to the specific training data used.

Building multiple models from samples of your training data, called bagging, can reduce this variance, but the trees are highly correlated.

Random Forest is an extension of bagging that in addition to building trees based on multiple samples of your training data, it also constrains the features that can be used to build the trees, forcing trees to be different. This, in turn, can give a lift in performance.

In this tutorial, you will discover how to implement the Random Forest algorithm from scratch in Python.

After completing this tutorial, you will know:
-  The difference between bagged decision trees and the random forest algorithm.
-  How to construct bagged decision trees with more variance.
-  How to apply the random forest algorithm to a predictive modelinng problem.

Let's get started.

<img src='files/img/randomforest.png'>

## Random Forest Algorithm

Decision trees involve the greedy selection of the best split point from the dataset at each step.

This algorithm makes decision trees susceptible to high variance if they are not pruned. This high variance can be harnessed and reduced by creating multiple trees with different samples of the training dataset (different views of the problem) and combining their predictions. This approach is called bootstrap aggregation or bagging for short.

A limitation of bagging is that the same greedy algorithm is used to create each tree, meaning that it is likely that the same or very similar split points will be chosen in each tree making the different trees very similar (trees will be correlated). This, in turn, makes their predictions similar, mitigating the variance originally sought.

We can force the decision trees to be different by limiting the features (rows) that the greedy algorithm can evaluate at each split point when creating the tree. This is called the Random Forest algorithm.

Like bagging, multiple samples of the training dataset are taken and a different tree trained on each. The difference is that at each point a split is made in the data and added to the tree, only a fixed subset of attributes can be considered.

For classification problems, the type of problems we will look at in this tutorial, the number of attributes to be considered for the split is limited to the square root of the number of input features.

$$\text{number of features for split} = \sqrt{\text{total input features}}$$

The result of this one small change are trees that are more different from each other (uncorrelated) resulting in predictions that are more diverse and a combined prediction that often has better performance than a single tree or bagging alone.

## Sonar Dataset

The dataset we will use in this tutorial is the Sonar dataset.

This is a dataset that describes sonar chirp returns bouncing off different surfaces. The 60 input variables are the strength of the returns at different angles. It is a binary classification problem that requires a model to differentiate rocks from metal cylinders. There are 208 observations.

It is a well-understood dataset. All of the variables are continuous and generally in the range of 0 to 1. The output variable is a string "M" for mine and "R" for rock, which will need to be converted to integers 1 and 0.

By predicting the class with the most observations in the dataset (M or mines) the Zero Rule Algorithm can achieve an accuracy of 53%.

You can learn more about this dataset at the [UCI Machine Learning repository.](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks))

Download the dataset for free and place it in your working directory with the filename __sonar.all-data.csv__.

## Tutorial

This tutorial is broken down into 2 steps.

1. Calculating Splits.
2. Sonar Dataset Case Study.

These steps provide the foundation that you need to implement and apply the Random Forest algorithm to your own predictive modeling problems.

### 1. Calculating Splits

In a decision tree, split points are chosen by finding the attribute and the value of that attribute that results in the lowest cost.

For classification problems, this cost function is often the Gini index, that calculates the purity of the groups of data created by the split point. A Gini index of 0 is perfect purity where class values are perfectly separated into two groups, in the case of a two-class classification problem.

Finding the best split point in a decision tree involves evaluating the cost of each value in the training dataset for each input variable.

For bagging and random forest, this procedure is executed upon a sample of the training dataset, made with replacement. Sampling with replacement means that the same row may be chosen and added to the sample more than once.

We can update this procedure for Random Forest. Instead of enumerating all values for input attributes in search of the split with the lowest cost, we can create a sample of the input attributes to consider.

This sample of input attributes can be chosen randomly and without replacement, meaning that each input attribute needs only be considered once when looking for the split point with the lowest cost.

Below is a function named __get_split()__ that implements this procedure. It takes a dataset and a fixed number of input features from which to evaluate as input arguments, where the dataset may be a sample of the actual training dataset.

The helper function __test_split()__ is used to split the dataset by a candidate split point and __gini_index()__ is used to evaluate the cost of a given split by the groups of rows created.

We can see that a list of features is created by randomly selecting feature indices and adding them to a list (called __features__), this list of features is then enumerated and specific values in the training dataset are evaluated as split points.

In [1]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
    # Get a list of all unique class values
    class_values = list(set(row[-1] for row in dataset))
    
    # Record best values
    b_index, b_value, b_gini, b_groups = 999, 999, 999, None
    
    # Randomly select n feature indices without replacement
    features = random.sample(range(len(dataset[0])-1), n_features)

    # Iterate over all the elements in the selected feature columns
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini   = gini_index(groups, class_values)
            
            # If the split has a lower gini score
            if gini < b_gini:
                b_index  = index
                b_value  = row[index]
                b_gini   = gini
                b_groups = groups
    
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

Now that we know how a decision tree algorithm can be modified for use with the Random Forest algorithm, we can piece this together with an implementation of bagging and apply it to a real-world dataset.

### 2. Sonar Dataset Case Study

In this section, we will apply the Random Forest algorithm to the Sonar dataset.

The example assumes that a CSV copy of the dataset is in the current working directory with the file name __sonar.all-data.csv__.

The dataset is first loaded, the string values converted to numeric and the output column is converted from strings to the integer values of 0 and 1. This is achieved with helper functions __load_csv()__.

We will use k-fold cross validation to estimate the performance of the learned model on unseen data. This means that we will construct and evaluate k models and estimate the performance as the mean model error. Classification accuracy will be used to evaluate each model. These behaviors are provided in the __cross_validation_split()__, __accuracy_metric()__ and __evaluate_algorithm()__ helper functions.

We will also use an implementation of the Classification and Regression Trees (CART) algorithm adapted for bagging including the helper functions __test_split()__ to split a dataset into groups, __gini_index()__ to evaluate a split point, our modified __get_split()__ function discussed in the previous step, __to_terminal()__, __split()__ and __build_tree()__ used to create a single decision tree, __predict()__ to make a prediction with a decision tree, __subsample()__ to make a subsample of the training dataset and __bagging_predict()__ to make a prediction with a list of decision trees.

A new function named __random_forest()__ is developed that first creates a list of decision trees from subsamples of the training dataset and then uses them to make predictions.

As we stated above, the key difference between Random Forest and bagged decision trees is the one small change to the way that trees are created, here in the __get_split()__ function.

The complete example is listed below.

In [2]:
# Random Forest Algorithm on Sonar Dataset
import random
import math

# Load a CSV file
def load_csv(filename):
    with open(filename) as f:
        dataset = [[x for x in line.split(',')] for line in f if line.strip()]
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = set([row[column] for row in dataset])
    lookup = {}
    for i, value in enumerate(class_values):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k-folds
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = sum([f for f in folds if f != fold], [])
        test_set = [row[:-1] + [None] for row in fold]
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split a dataset based on an attribute and a split value
def test_split(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    
    # Count total samples
    total_samples = float(sum([len(group) for group in groups]))
    
    # Sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        
        # Avoid divide by zero
        if size == 0:
            continue
        
        # Score the group based on the score for each class
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p**2
            
        # Weight the groups score by its relative size
        gini += (1.0 - score) * (size / total_samples)
        
    return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
    # Get a list of all unique class values
    class_values = list(set(row[-1] for row in dataset))
    
    # Record best values
    b_index, b_value, b_gini, b_groups = 999, 999, 999, None
    
    # Randomly select n feature indices without replacement
    features = random.sample(range(len(dataset[0])-1), n_features)

    # Iterate over all the elements in the selected feature columns
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini   = gini_index(groups, class_values)
            
            # If the split has a lower gini score
            if gini < b_gini:
                b_index  = index
                b_value  = row[index]
                b_gini   = gini
                b_groups = groups
    
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # Check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth + 1)
        
    # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth + 1)
        
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if isinstance(node, dict):
        if row[node['index']] < node['value']:
            return predict(node['left'], row)
        else:
            return predict(node['right'], row)
    else:
        return node
    
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    n_sample = round(len(dataset) * ratio)
    sample = random.choices(dataset, k=n_sample)
    return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_ratio, n_trees, n_features):
    samples = [subsample(train, sample_ratio) for _ in range(n_trees)]
    trees = [build_tree(sample, max_depth, min_size, n_features) for sample in samples]
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions

if __name__ == '__main__':
    # Load data
    dataset = load_csv('data/sonar.all-data.csv')
    for i in range(len(dataset[0])-1):
        str_column_to_float(dataset, i)
    str_column_to_int(dataset, -1)
    
    # Evaluate algorithm
    n_folds = 5
    max_depth = 10
    min_size = 1
    sample_ratio = 1.0
    n_features = int(math.sqrt(len(dataset[0])-1))
    
    for n_trees in [1, 5, 10]:
        scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_ratio, n_trees, n_features)
        print('Trees: {0}'.format(n_trees))
        print('Scores: {0}'.format(scores))
        print('Mean Accuracy: {0}%\n'.format(sum(scores) / float(len(scores))))

Trees: 1
Scores: [68.29268292682927, 73.17073170731707, 65.85365853658537, 68.29268292682927, 56.09756097560976]
Mean Accuracy: 66.34146341463415%

Trees: 5
Scores: [65.85365853658537, 78.04878048780488, 78.04878048780488, 80.48780487804879, 75.60975609756098]
Mean Accuracy: 75.60975609756098%

Trees: 10
Scores: [70.73170731707317, 87.8048780487805, 75.60975609756098, 87.8048780487805, 75.60975609756098]
Mean Accuracy: 79.51219512195124%



A _k_ value of 5 was used for cross-validation, giving each fold 208/5 = 41.6 or just over 40 records to be evaluated upon each iteration.

Deep trees were constructed with a max depth of 10 and a minimum number of training rows at each node of 1. Samples of the training dataset were created with the same size as the original dataset, which is a default expectation for the Random Forest algorithm.

The number of features considered at each split point was set to sqrt(num_features) or sqrt{60} = 7.74 rounded to 7 features.

A suite of 3 different numbers of trees were evaluated for comparison, showing the increasing skill as more trees are added.

## Extensions

This section lists extensions to this tutorial that you my be interested in exploring.

-  __Algorithm Tuning.__ The configuration used in the tutorial was found with a little trial and error but was not optimized. Experiment with larger number of trees, different numbers of features and even different tree configurations to improve performance.
-  __More Problems.__ Apply the technique to other classification problems and even adapt it for regression with a new cost function and a new method for combining the predictions from trees.